In [81]:
import pandas as pd
from collections import deque

In [83]:
# Import future ENT appointments
data = pd.read_csv("../future_appts_trans_ENT.csv")
data

,appointment_id,age_deid,bmi_deid,ethnic_category_black,ethnic_category_unknown,gender_male,imd19_quintile,main_spoken_language_other,current_smoker,substance_misuse,...,appt_date,ethnicity_trans,tfc_name_trans,patient_on_multiple_pathways_trans,appt_month_trans,hospital_service_display_trans,service_mapping_trans,reason_display_trans,appointment_type_trans,did_not_attend_risk_group_trans
0,17479744,57,27.45,1,0,1,1,0,0,0,...,2024-06-26,3,7,0,6,0,10,1,0,3
1,17631073,17,27.45,0,1,0,2,0,0,0,...,2024-05-07,3,7,0,5,0,10,1,0,3
2,17659262,27,21.70,0,0,0,4,0,0,0,...,2024-05-07,5,7,0,5,0,10,2,0,2
3,17659348,57,21.70,0,0,0,3,0,0,0,...,2024-05-07,5,7,0,5,0,10,2,0,3
4,17659361,27,27.45,0,1,0,2,0,0,0,...,2024-05-07,3,7,0,5,0,10,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560,20183628,47,27.45,0,0,1,2,0,0,0,...,2024-06-19,5,7,0,6,0,10,1,0,1
2561,20183649,57,40.00,0,1,0,1,0,0,0,...,2024-06-08,3,7,0,5,0,10,2,0,3
2562,20183782,32,21.70,0,0,0,4,0,0,0,...,2024-05-25,3,7,0,7,0,10,6,0,1
2563,20183795,62,39.45,0,0,0,1,0,0,0,...,2024-06-17,3,7,0,6,0,10,3,0,1


In [84]:
# Filter for surgeons & new appointments only
surgeon_new = data[data["reason_display"].isin(["Adult ENT New", "Adult ENT Thyroid New"])]

In [95]:
# Within the surgeons & new appointments only, filter for very high risk appointments
surgeon_new_very_high = surgeon_new[surgeon_new["did_not_attend_risk_group_trans"] == 4]

In [92]:
# Count number of unique appointment durations
surgeon_new["appt_duration"].value_counts()

appt_duration
15.0    1962
30.0     198
20.0      33
10.0      17
75.0       3
Name: count, dtype: int64

In [96]:
# Split out the appointment durations
surgeon_new_10 = surgeon_new[surgeon_new["appt_duration"] == 10]
surgeon_new_15 = surgeon_new[surgeon_new["appt_duration"] == 15]
surgeon_new_20 = surgeon_new[surgeon_new["appt_duration"] == 20]
surgeon_new_30 = surgeon_new[surgeon_new["appt_duration"] == 30]
surgeon_new_75 = surgeon_new[surgeon_new["appt_duration"] == 75]

# Split out the very high risk appointments
surgeon_new_very_high_10 = surgeon_new_very_high[surgeon_new_very_high["appt_duration"] == 10]
surgeon_new_very_high_15 = surgeon_new_very_high[surgeon_new_very_high["appt_duration"] == 15]
surgeon_new_very_high_20 = surgeon_new_very_high[surgeon_new_very_high["appt_duration"] == 20]
surgeon_new_very_high_30 = surgeon_new_very_high[surgeon_new_very_high["appt_duration"] == 30]
surgeon_new_very_high_75 = surgeon_new_very_high[surgeon_new_very_high["appt_duration"] == 75]

In [86]:
# # Group weekswaiting as per NHS guidelines
# data["weekswaiting"]
# data["weekswaiting_grouped"] = pd.cut(data["weekswaiting"], bins=[0, 17, 51, 76, float('inf')], labels=["0-17", "18-51", "52-77", ">78"])
# data[["weekswaiting","weekswaiting_grouped"]]

### V1: overbook any into very high risk based on weeks waiting

In [101]:
# Sort the appointments by weekswaiting (descending; all except the very high risk ones)
weekswaiting_desc = surgeon_new[surgeon_new["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)
    # ? Create a new df with >78 weeks waiting - prioritise these patients; but if < # of slots, then fill with 18-52 week patients?

# Sort the individual appointment durations by weekswaiting (descending; all except the very high risk ones)
weekswaiting_desc_10 = surgeon_new_10[surgeon_new_10["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)
weekswaiting_desc_15 = surgeon_new_15[surgeon_new_15["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)
weekswaiting_desc_20 = surgeon_new_20[surgeon_new_20["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)
weekswaiting_desc_30 = surgeon_new_30[surgeon_new_30["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)
weekswaiting_desc_75 = surgeon_new_75[surgeon_new_75["did_not_attend_risk_group_trans"] != 4].sort_values(by="weekswaiting", ascending=False)



In [102]:
# Create a queue for the remaining appointments
queue = deque(weekswaiting_desc["appointment_id"])
queue10 = deque(weekswaiting_desc_10["appointment_id"])
queue15 = deque(weekswaiting_desc_15["appointment_id"])
queue20 = deque(weekswaiting_desc_20["appointment_id"])
queue30 = deque(weekswaiting_desc_30["appointment_id"])
queue75 = deque(weekswaiting_desc_75["appointment_id"])


# Count
len(queue)

1974

In [89]:
overbooked_appointments = []
for very_high_appt in surgeon_new_very_high["appointment_id"]:
    if queue:
        earliest_appointment = queue.popleft()
        # Overbook the high risk slot with the remaining appointment
        overbooked_appointments.append((very_high_appt, earliest_appointment))
    else:
        # No more appointments to overbook
        overbooked_appointments.append((very_high_appt, None))

overbooked_appointments

[(17659386, 18843556),
 (17671208, 19238262),
 (17900340, 17667343),
 (17901062, 17667391),
 (18011196, 17671153),
 (18012765, 17671162),
 (18017248, 17671200),
 (18024409, 17671294),
 (18043624, 20112378),
 (18132699, 17659262),
 (18148527, 17659348),
 (18222404, 17659361),
 (18247682, 17659378),
 (18258255, 17659394),
 (18266354, 20026346),
 (18266513, 20026219),
 (18266700, 18878690),
 (18270101, 17742418),
 (18281582, 20002224),
 (18312755, 19718639),
 (18369259, 18000683),
 (18372280, 17747054),
 (18379094, 17747532),
 (18384272, 17819013),
 (18384602, 19971818),
 (18384704, 19826622),
 (18407248, 20025984),
 (18483017, 17849583),
 (18483391, 17849809),
 (18504355, 17848543),
 (18505702, 17900599),
 (18518462, 17900741),
 (18519612, 17900724),
 (18523745, 17855191),
 (18537978, 17900647),
 (18557075, 17854821),
 (18559029, 17900868),
 (18559411, 17900585),
 (18561212, 17900419),
 (18571725, 17900956),
 (18576052, 17901100),
 (18606091, 20111091),
 (18606390, 18825605),
 (18606778,

In [90]:
# Print remaining appointments after overbooking
print("\nRemaining appointments:")
for appt in queue:
    print(appt)

# Count 
len(queue)


Remaining appointments:
18355893
18267845
18270131
18270183
18270481
18270549
18270772
18270705
18370162
18369330
18356541
18369162
18369175
18370317
18370284
18370247
18369272
18380143
18381618
18409962
18381758
18382926
18383816
18373002
18373155
18371768
18296387
18371914
18372263
18372926
18374467
18378582
18373513
18373227
18374450
18419518
18379081
19972085
18379162
18375372
20002561
19939772
18374636
18374590
18374563
18375359
20024565
18385448
18385318
18385804
18385536
18384388
18385380
18385356
18385336
18385306
18385909
18385240
18384749
18384739
18384682
18384672
18384550
18385901
18385920
18384459
18389282
18379492
20024567
18383911
18383942
18383964
18389493
18389272
18384172
18389196
18389185
18389150
18389039
18384073
18388968
18384521
18519566
18519622
18514589
18515659
18520597
18520547
18519767
18519747
18519757
18468241
20024775
18482806
18411129
18411109
18411084
18411049
18411013
18410730
18410681
18410652
18410645
18410634
18410614
18410244
18410051
18407198
183

1735

In [104]:
# Function to overbook appointments
def overbook_appointments(very_high_group, queue):
    overbooked_appointments = []
    for index, very_high_appt in very_high_group.iterrows():
        if queue:
            earliest_appointment_id = queue.popleft()
            overbooked_appointments.append((very_high_appt["appointment_id"], earliest_appointment_id))
        else:
            overbooked_appointments.append((very_high_appt["appointment_id"], None))
    return overbooked_appointments


# Overbook each very high risk appointment group
overbooked_10 = overbook_appointments(surgeon_new_very_high_10, queue10)
overbooked_15 = overbook_appointments(surgeon_new_very_high_15, queue15)
overbooked_20 = overbook_appointments(surgeon_new_very_high_20, queue20)
overbooked_30 = overbook_appointments(surgeon_new_very_high_30, queue30)
overbooked_75 = overbook_appointments(surgeon_new_very_high_75, queue75)

# Combine all overbooked results
all_overbooked_appointments = overbooked_10 + overbooked_15 + overbooked_20 + overbooked_30 + overbooked_75

# Convert to DataFrame
overbooked_df = pd.DataFrame(all_overbooked_appointments, columns=['very_high_risk_appointment_id', 'overbooked_appointment_id'])

